In [11]:
require('nn')
require('hdf5')
require('optim')

In [12]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

nSen = 41931
lenSen = 85

window_size = X_train:size(2)
n_actual = nfeatures

In [13]:
function idxSen(X, n)
    local sen_count = 0
    local new_sen_idx = torch.zeros(1, n)
    for i=1,X:size(1) do
        local row = X[i]
        if (row[1] ==row[2] and row[2]==row[3] and row[1]==3) then
            sen_count = sen_count + 1
            new_sen_idx[1][sen_count] = i 
        end
    end
    return new_sen_idx
end

function senMatrix(X, new_sen_idx, sen_len)
    local sen_counter = 2
    local i = 1
    local c = 0
    local sen_matrix = torch.zeros(sen_count, sen_len)
    while i < X:size(1) do
        if i < new_sen_idx[1][sen_counter] then
            for j=1,5 do
                c = c + 1
                sen_matrix[sen_counter-1][c]= X[i][j]
            end
            i = i + 5
        else
            local last_win = new_sen_idx[1][sen_counter]-1
            local mid_sen = (2+5-(new_sen_idx[1][sen_counter]-(i-5)))
            for j=mid_sen,5 do
                c = c + 1
                sen_matrix[sen_counter-1][c] = X[last_win][j]
            end
            c = 0
            i = new_sen_idx[1][sen_counter]
            sen_counter = sen_counter + 1
            if sen_counter > new_sen_idx:size(2) then
                break
            end
        end
    end
    return sen_matrix
end

In [14]:
sen_idx = idxSen(X_train, nSen)
sen_matrix = senMatrix(X_train, sen_idx, lenSen)

In [15]:
sen_matrix:sub(10,10)

Columns 1 to 13
    3     3     3    66   279  1924    45    74   197   159  1454  2379     6

Columns 14 to 26
 3222   717   108  5839  1299     0     0     0     0     0     0     0     0

Columns 27 to 39
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 40 to 52
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 53 to 65
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 66 to 78
    0     0     0     0     0     0     0     0     0     0     0     0     0

Columns 79 to 85
    0     0     0     0     0     0     0
[torch.DoubleTensor of size 1x85]

